<a href="https://colab.research.google.com/github/Sangeetha2998/Ask-emily/blob/main/emil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (3,114 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initial

In [2]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [3]:
!pip install -U lightrag[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 14.6 MB/s eta 0:00:00


In [1]:
!pip install streamlit
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.4 MB/s eta 0:00:00


In [5]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import time
from IPython.display import Markdown, display

# Updated QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User  {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

# Initialize the model client for Ask Emily
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Example user query relevant to the Ask Emily system
user_query = "Can you recommend a scenic route for RV travel from Los Angeles to San Francisco with good campgrounds along the way?"
output = qa.call(user_query)

# Display the output in Markdown format
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** I'd be happy to help you plan a scenic route for your RV trip from Los Angeles to San Francisco.

One of my top recommendations is to take the Pacific Coast Highway (PCH), also known as US-101, which runs along the California coast. This route offers breathtaking ocean views, charming coastal towns, and some of the most beautiful campgrounds in the state.

Here's a suggested itinerary:

Day 1: Los Angeles to Cambria

* Depart from Los Angeles and head north on PCH (US-101)
* Stop at Big Sur for stunning coastal views and a visit to Hearst Castle (book your tickets in advance!)
* Continue driving and arrive at Cambria, a quaint coastal town with art galleries, shops, and excellent campgrounds like El Colibri Hotel & Gardens or Moonstone Landing RV Park

Day 2: Cambria to Monterey

* Depart from Cambria and continue on PCH (US-101)
* Stop in Morro Bay for lunch and some beach time
* Visit the famous Monterey Bay Aquarium and enjoy the scenic coastal views along Cannery Row
* Camp at a nearby RV park like El Toro Lodge or Monterey Pines RV Resort

Day 3: Monterey to Half Moon Bay

* Depart from Monterey and continue on PCH (US-101)
* Stop in Santa Cruz for some beach time and a visit to the famous Santa Cruz Beach Boardwalk
* Continue driving and arrive at Half Moon Bay, a picturesque coastal town with beautiful beaches and excellent campgrounds like Half Moon Bay State Beach or Sunset State Beach

Day 4: Half Moon Bay to San Francisco

* Depart from Half Moon Bay and head east on PCH (US-101)
* Stop in Pacifica for lunch and some beach time
* Arrive in San Francisco, where you can explore the city's famous landmarks like Fisherman's Wharf, Golden Gate Bridge, and Alcatraz Island.

This route offers a mix of coastal scenery, charming towns, and excellent campgrounds. However, keep in mind that this is just one of many options, and I'd be happy to suggest alternative routes or campgrounds based on your preferences.

Would you like me to recommend any specific RV-friendly campgrounds along the way? Or perhaps provide more information on RV maintenance and repair services near these campsites?

npm WARN using --force Recommended protections disabled.
npm ERR! code ENOLOCK
npm ERR! audit This command requires an existing lockfile.
npm ERR! audit Try creating one first with: npm i --package-lock-only
npm ERR! audit Original error: loadVirtual requires existing shrinkwrap file

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-11-07T14_50_22_756Z-debug-0.log


In [10]:
# Install necessary libraries
!pip install streamlit lightrag
!npm init -y
!npm install -g localtunnel

!npm install --package-lock-only

!npm audit fix
# Create the Streamlit app
%%writefile app.py
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
import streamlit as st

# Updated QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User   {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

# Initialize the model client for Ask Emily
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Streamlit app layout
st.title("Ask Emily - Your Travel Assistant")
st.write("Welcome! Ask me anything about trip planning, RV navigation, or finding RV technicians.")

# Initialize chat history
if 'messages' not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Hello! How can I assist you today?"}]

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# User input
if user_input := st.chat_input("You:"):
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.write(user_input)

    # Generate and display the assistant's response
    output = qa.call(user_input)
    st.session_state.messages.append({"role": "assistant", "content": output.data})
    with st.chat_message("assistant"):
        st.write(output.data)

Wrote to /content/package.json:

{
  "name": "content",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}



changed 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.

up to date, audited 1 package in 138ms

found 0 vulnerabilities

up to date, audited 1 package in 138ms

found 0 vulnerabilities


UsageError: Line magic function `%%writefile` not found.
